In [ ]:
import socket
import base64
from io import BytesIO
from PIL import Image
import cv2
import numpy as np
import os
import sys
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib

In [ ]:
#adjust images
def image_to_base64(image_path):
    img = Image.open(image_path).convert('RGB')
    output_buffer = BytesIO()
    img.save(output_buffer, format='JPEG')
    byte_data = output_buffer.getvalue()
    base64_str = base64.b64encode(byte_data)
    return base64_str
def base64_to_image(base64_str, image_path=None):
    byte_data = base64.b64decode(base64_str)
    image_data = BytesIO(byte_data)
    img = Image.open(image_data)
    if image_path:
        img.save(image_path)
    return img

In [ ]:
# if the rgb of mask area is over 185 then return false
def video_check_light(im,mask,gate):
    flag=True
    img=cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    img = np.array(img)
    count=0
    for i,j in zip(np.where(mask == 1)[0],np.where(mask == 1)[1]):
        count+=img[i,j].sum()
    print((count/3)/np.where(mask == 1)[0].shape[0])
    if ((count/3)/np.where(mask == 1)[0].shape[0])>=gate:
        flag=False
    return flag


def random_colors(N):
    np.random.seed(1)
    colors = [tuple(255 * np.random.rand(3)) for _ in range(N)]
    return colors


def apply_mask(image, mask, color, alpha=0.5):
    """apply mask to image"""
    for n, c in enumerate(color):
        image[:, :, n] = np.where(
            mask == 1,
            image[:, :, n] * (1 - alpha) + alpha * c,
            image[:, :, n]
        )
    return image

## Calculate area here

In [ ]:
def display_instances(image, boxes, masks, ids, names, scores):
    """
        take the image and results and apply the mask, box, and Label
    """
    n_instances = boxes.shape[0]
    colors = random_colors(n_instances)

    if not n_instances:
        print('NO INSTANCES TO DISPLAY')
    else:
        assert boxes.shape[0] == masks.shape[-1] == ids.shape[0]


    pixel_clip=-1 #Initialize the true area of each point
    for count, color in enumerate(colors):
        if names[ids[count]]=="clip":
            y1, x1, y2, x2 = boxes[count]
            side_length_pixel=7*7/((x2-x1)**2+(y2-y1)**2)
            pixel_clip = side_length_pixel         #true area of each point
            
            diagonal = ((x2-x1)**2+(y2-y1)**2)**0.5
            true_diagonal = 7/diagonal
            break


    for i, color in enumerate(colors):
        if not np.any(boxes[i]):
            continue

        y1, x1, y2, x2 = boxes[i]
        label = names[ids[i]]

        if label == "hole":
            continue
        
        score = scores[i] if scores is not None else None


        mask = masks[:, :, i]
        total_area = mask.shape[0]*mask.shape[1] #total area of image
        mask_percentage = np.count_nonzero(mask)/total_area

        diagonal_label=""
        area_label=""
        if pixel_clip==-1 and label == "polyp":     #No clips show how % of the whole picture   
            area_label="areaperc:" + str(round(mask_percentage*100,4))+"%"
        elif pixel_clip!=-1 and label == "polyp":   #with clip then show mm2
            polyp_diagonal = true_diagonal*((x2-x1)**2+(y2-y1)**2)**0.5
            area_label="realarea:" + str(round(np.count_nonzero(mask)*pixel_clip,2))+" mm2"
            diagonal_label="diagonal=" + str(round(polyp_diagonal,2)) + "mm"

        label = label + " conf:"
        caption = '{} {:.2f}'.format(label, score) if score else label


        if video_check_light(image,mask,gate=125):  #use video_check_light if don't needed just delete
            image = apply_mask(image, mask, color)
            image = cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
            image = cv2.putText(
                image, caption, (x1, y1-44), cv2.FONT_HERSHEY_COMPLEX, 0.7, color, 2
            )
            image = cv2.putText(
                image, area_label, (x1, y1-23), cv2.FONT_HERSHEY_COMPLEX, 0.7, color, 2
            )
            image = cv2.putText(
                image, diagonal_label, (x1, y1-2), cv2.FONT_HERSHEY_COMPLEX, 0.7, color, 2
            )
    return image

## Config

In [ ]:
ROOT_DIR = os.getcwd()
MODEL_DIR = os.path.join(ROOT_DIR, "logs")
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "logs\\your_model.h5")


class InferenceConfig(Config):
    NAME = "polyp"
    NUM_CLASSES = 1 + 2
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    IMAGE_MIN_DIM = 512
    IMAGE_MAX_DIM = 512
    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)
    TRAIN_ROIS_PER_IMAGE =100
    STEPS_PER_EPOCH = 200
    VALIDATION_STEPS = 50

config = InferenceConfig()
config.display()

model = modellib.MaskRCNN(
    mode="inference", model_dir=MODEL_DIR, config=config
)
model.load_weights(COCO_MODEL_PATH, by_name=True)

# Class names firsth one should be BG background
class_names = ['BG','polyp', 'clip']  #'hole']

## Start the server

In [ ]:
HOST = '10.1.202.132'  #host ip address
PORT = 8000

server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server.bind((HOST, PORT))
server.listen(10)

while True:
    conn, addr = server.accept()
    recieve  = conn.recv(99999999999)
    img = base64_to_image(recieve)     #change image format
    
    #predict
    ##################################################################
    results = model.detect([np.array(img)], verbose=0)
    r = results[0]
    frame = display_instances(
        np.array(img), r['rois'], r['masks'], r['class_ids'], class_names, r['scores']
    )
    Image.fromarray(frame).save("temp.jpg")     #save the result
    ##################################################################
    
    conn.sendall(image_to_base64("temp.jpg"))#serverMessage.encode())
    conn.close()

#server.close()